<h1 style="text-align: center;" markdown="1">Rapport TAL: Classification de documents par rapport à leur type de discours à l'aide de l'analyse du discursive</h1>


<h3 style="text-align: right;" markdown="1">René Traoré, de Bézenac Emmanuel</h3>



## Introduction

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;L’analyse du discours dans le contexte du traitement automatique du langage a pour but une compréhension plus fine et plus en profondeur du texte en tentant de dégager une structure discursive dans celui-ci. Certaines subtilités sont difficilement détectables avec la plupart des modèles utilisés en traitement automatique du langage, qui se basent essentiellement sur des traits surfaciques comme le nombre d’occurrences d’un mot dans un document par exemple. Prenons un exemple qui illustre l’importance de pouvoir exploiter la structure discursive d’un texte trouvé dans [NLPERS]:

    1: J’aime uniquement voyager en Europe. Alors j’ai soumis un article à ACL.
    2. J’aime uniquement voyager en Europe. Néanmoins j’ai soumis un article à ACL. 

<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Lorsque ces deux couples de phrases sont traités indépendemment, on peut inférer les mêmes connaissances: Il aime voyager en Europe, et il a soumis un article à ACL. Mais grâce à l’analyse du discours nous pouvons inférer davantage d’informations, à savoir, en 1: ACL se déroule en Europe, et en 2:  ACL n’est pas en Europe. </br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Plus concrètement, l’analyse discursive vise d’abord à segmenter le texte en unités élémentaire du discours (EDU),  puis à les rattacher, et identifier leur type de relation, puis récursivement les paires attachées sont liées à des segments simples ou complexes pour aboutir à une structure couvrant le document. Voici un exemple donné par [Braud, Denis 2013]:

    Exemple 1.1: {{[La hulotte est un rapace nocturne] [mais elle peut vivre le jour.]} contrast [La hulotte mesure une quarantaine de centimètres.]} continuation

    Exemple 1.2: {[Juliette est tombée.] [Marion l’a poussée.]} explanation
    
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Le texte entre crochets représente un EDU, tandis que les accolades représentent l’existence d’une relation entre la paire de segments contenue par les accolades. Notons qu’il est possible d’avoir des relations entre EDU, mais également entre paires d’EDU, et ceci récursivement.

&nbsp;


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Un type de discours est une forme de langage utilisé par l’émetteur (énonciateur) en fonction du
but qu’il vise et du message qu’il souhaite transmettre à ses lecteurs. L’émetteur pourrait
vouloir :
* Raconter une histoire : dans ce cas, il utiliserait le **discours narratif**
* Expliquer un fait ou donner une information : il utiliserait le **discours explicatif** 
* Essayer de convaincre ou de faire changer d’opinion à ses lecteurs sur un sujet précis : **discours argumentatif**



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Il pourrait être intéressant de voir si une corrélation entre analyse du discours et type de discours existe, et si oui, comment elle se traduit. Ce projet à pour but d'étudier cette question, en construisant des représentations latentes de documents  basées sur l'analyse du discours pour tenter de prédire le type de discours associé à celui-ci.



## I - Données - Constitution du corpus

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Faute d'avoir trouvé un corpus étiqueté préexistant pour notre tache de classification, nous avons du en consituer un. Nous avons dû nous poser quelques hypothèses simplificatrices pour pouvoir récupérer des documents en taille et en nombre suffisant. Nous savons que plusieurs types de discours peuvent être présents dans un seul texte au même moment. Nous faisons l'hypothèse qu'il n'y a qu'un seul pour un document. Nous supposons également que les documents intra-classe ne sont pas extraits de sources hétérogènes, sont représentatifs de la classe à laquelle ils appartiennent.




* **narratif**: constitué de la partie "Plot" dans des pages de films sur wikipedia.org.
* **argumentatif**: constitué des en-têtes des pages accessibles depuis la page "Science" de wikipedia.org. 
* **informatif**: consitué des dicours de présidents des États-Unis, accessibles depuis millercenter.org.

| Type de dicours    |     Taille moyenne (mots) |   Nombre     |  Origine |
| :-------------      | :---------------: | :---------:  | :---------: |
| **narratif**       |      694          |   731         | wikipedia.org|
| **argumentatif**   |      935          |   192         | wikipedia.org|
| **informatif**     |      400          |   226         | millercenter.org|


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Nous pouvons voir que les données inter-classe sont très hétérogènes. Il faudra en tenir compte durant la procédure d'évaluation, afin de ne pas biaiser la prédiction. Pour les données extraites depuis wikipedia.org, nous avons utilisé une API *wikipedia* pour Python. Pour les données de type argumentatives, il a fallu créer un webscrapper à l'aide de *BeautifulSoup*, qui extrait des liens à partir de la page principale, pour ensuite récupérer et filtrer l'information contenue sur la page hmtl.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Nous avons ensuite effectué un pre-processing sur ces données: élimination des caractères inconnus, et application d'une expression régulière afin de segmenter le texte en phrases.


## II - Du texte à la représentation


### Structure discursive 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Pour appliquer les modèles de classification, il faut pouvoir représenter un document, en le projetant dans un espace latent. Une méthode très utilisée dans le traitement automatique du langage est de projeter les documents dans un espace caractérisé par le vocabulaire utilisé. Cette technique, dite *Bag of Words* peut s'avérer très utile pour certains cas de figure, mais on lui reproche souvent de ne pas tenir compte de l'ordre des mots dans le document, et de ne pas pouvoir capter la sémantique de ce dernier. Le vocabulaire utilisé dans un texte d'un type de discours particulier semble ne pas le caractériser suffisamment, même si l'on pourrait s'attendre à pouvoir extraire quelques informations pertinentes, des traits grammaticaux discriminants pour un type de texte particulier. Il pourrait donc être intéressant d'utiliser l'analyse du discours qui fait une analyse plus "profonde" du document pour trouver des traits plus pertinents et plus représentatifs de la classe des documents, mais aussi et surtout de la distinction entre ces derniers.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Nous avons donc choisi d'utiliser un parseur *DPLP* *Representation Learning for Text-level Discourse Parsing*, de *Yangfeng Ji*, et de *Jacob Eisenstein*, étant l'état de l'art en terme de parseur discursif (disponible depuis https://github.com/jiyfeng/DPLP). reconstitue la structure de dépendances des éléments de discours élémentaire, avec ainsi que les relations associés, structure qui se traduit sous forme d'arbre binaire.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Mais avant de pouvoir utiliser l'utiliser, il faut appliquer des pré-traitements aux documents:
    * POS-tagging
    * lemmatisation
    * Named Entity Recognizer
Pour cela, nous avons utilisé CoreNLP de Stanford, une suite d'outils pour le traitement automatique de la langue, accessible depuis le lien suivant: http://stanfordnlp.github.io/CoreNLP.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Une fois CoreNLP et DPLP appliqués sur notre corpus, nous avons une structure arborescente par document. Nous avons modifié l'algorithme DPLP pour pouvoir renvoyer des arbres au format *nltk.Tree*. Voici une visualisation d'un arbre après traitements:

![arbre](images/arbre.png)


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Comme avec les *Bags of Words*, nous avons besoin d'une représentation à donner à nos classifieurs. La première représentation choisie reprend le concept de la représentation *Bag of Words*, mais au lieu de prendre le vocabulaire caractériser notre espace, nous utilisons les différentes relations présentes dans un document. Voici un exemple plus concret:

|Représentation du doc X    | NS-elaboration |   SN-temporal    |  SN-purpose |  NN-textualorg | NS-purpose |
| :-------------      | :---------------: | :---------:  | :---------: |:---------: |:---------: |
| **bin**      |      0          |   1         | 1|  0 |  0 |
| **count**   |      0          |   4         | 6|  0 |  0 |
| **count_norm**     |      0          |   0.4   | 0.6|  0 |  0 |
| **tfid**     |      0          |   0.7         | 0.3|  0 |  0 |
| **count_height**     |    0     |   10   | 40  | 0  | 0  |
| **count_height_norm**     |    0     |   0.1   | 0.4 | 0  | 0  |


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Nous sommes donc passés d'une représentation arborescente d'un texte à une représentation vectorielle. Le vecteur **bin** contient un 1 sur la composante i si la relation est présente dans le document, et 0 sinon. Le vecteur **count** compte le nombre d'occurences des différentes relations. Le vecteur **count_norm** est la version normalisée pour que la somme des composantes soit égale à 1. De cette manière, nous avons une représentation qui est indépendante de la taille du document. **count_height** 

### Features additionnels








## III -  Noyaux et Modèles

Voici un récapitualif du pipeline mis en place:
![pipeline](images/pipeline.jpg)

## IV - Résultats, Interprétations

## Conclusion